In [1]:
# Modelos de Detección de Outliers
import plotly.graph_objects as go
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.ensemble import IsolationForest

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# 1. ISOLATION FOREST
def detect_with_isolation_forest(data, contamination=0.025):
    df = data.copy()
    
    # Preparar datos
    df_iso = df.drop(["Cliente", "Fecha"], axis=1)
    df_iso = df_iso.replace([np.inf, -np.inf], np.nan).ffill()
    
    # Escalar datos
    scaler = RobustScaler()
    X = scaler.fit_transform(df_iso)
    
    # Entrenar modelo
    clf = IsolationForest(n_estimators=250, contamination=contamination, random_state=42)
    df["anomaly"] = clf.fit_predict(X)
    df["anomaly"] = df["anomaly"].map({1: 0, -1: 1})  # Convertir a 0=normal, 1=anomalía
    
    return df

In [3]:
def detect_with_isolation_forest(df, contamination=0.1):
    # Eliminar filas con valores faltantes
    df = df.dropna()
    
    # Suponiendo que las columnas numéricas son las que vamos a usar para el modelo
    X = df.select_dtypes(include=[float, int])  # Ajusta según tus columnas
    
    # Entrenar modelo
    clf = IsolationForest(n_estimators=250, contamination=contamination, random_state=42)
    df["anomaly"] = clf.fit_predict(X)
    
    # Convertir a 0=normal, 1=anomalía
    df["anomaly"] = df["anomaly"].map({1: 0, -1: 1})
    
    return df


In [21]:
from sklearn.impute import SimpleImputer

def detect_with_isolation_forest(df, contamination=0.1):
    # Imputar valores NaN con la media
    imputer = SimpleImputer(strategy='mean')  # Puedes cambiar 'mean' por 'median' si prefieres la mediana
    df_imputed = pd.DataFrame(imputer.fit_transform(df.select_dtypes(include=[float, int])))
    
    # Mantener las columnas originales en el dataframe
    df[df_imputed.columns] = df_imputed
    
    # Suponiendo que las columnas numéricas son las que vamos a usar para el modelo
    X = df.select_dtypes(include=[float, int])  # Ajusta según tus columnas
    
    # Entrenar modelo
    clf = IsolationForest(n_estimators=250, contamination=contamination, random_state=42)
    df["anomaly"] = clf.fit_predict(X)
    
    # Convertir a 0=normal, 1=anomalía
    df["anomaly"] = df["anomaly"].map({1: 0, -1: 1})
    
    return df


In [4]:
def detect_with_isolation_forest(df, contamination=0.1):
    # Seleccionar solo columnas numéricas
    numeric_cols = df.select_dtypes(include=[float, int]).columns
    X = df[numeric_cols].copy()

    # Reemplazar inf y -inf por NaN
    X.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Imputar valores NaN con la media
    imputer = SimpleImputer(strategy='mean')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=numeric_cols)

    # Aplicar Isolation Forest
    model = IsolationForest(contamination=contamination, random_state=42)
    model.fit(X_imputed)
    df['Volumen_outlier_zscore'] = model.predict(X_imputed)  # -1 = outlier, 1 = normal
    df['Volumen_outlier_zscore'] = df['Volumen_outlier_zscore'].map({1: 0, -1: 1})
    
    return df


In [7]:
from sklearn.impute import SimpleImputer


In [8]:
import pandas as pd
from sklearn.impute import SimpleImputer  # Agregado
# Otros imports necesarios según tu función detect_with_isolation_forest

# Leer archivo consolidado Excel
df_total = pd.read_excel("preprocessingdata.xlsx", engine="openpyxl", parse_dates=["Fecha"])

# Crear lista para guardar resultados
resultados = []

# Procesar por cliente
for cliente, df_cliente in df_total.groupby('Cliente'):
    print(f"Procesando cliente: {cliente}")
    
    # Aplicar Isolation Forest
    df_iforest = detect_with_isolation_forest(df_cliente)
    
    # Añadir columna de nombre del cliente
    df_iforest["Cliente"] = cliente
    
    # Guardar resultados
    resultados.append(df_iforest)

# Combinar todos los resultados en un único DataFrame
df_resultado = pd.concat(resultados, ignore_index=True)

# Guardar como archivo Excel consolidado
df_resultado.to_excel("processingdata_IF.xlsx", index=False)
print("\n✅ Archivo 'processingdata_IF.xlsx' generado exitosamente.")


Procesando cliente: CLIENTE1
Procesando cliente: CLIENTE10
Procesando cliente: CLIENTE11
Procesando cliente: CLIENTE12
Procesando cliente: CLIENTE13
Procesando cliente: CLIENTE14
Procesando cliente: CLIENTE15
Procesando cliente: CLIENTE16
Procesando cliente: CLIENTE17
Procesando cliente: CLIENTE18
Procesando cliente: CLIENTE19
Procesando cliente: CLIENTE2
Procesando cliente: CLIENTE20
Procesando cliente: CLIENTE3
Procesando cliente: CLIENTE4
Procesando cliente: CLIENTE5
Procesando cliente: CLIENTE6
Procesando cliente: CLIENTE7
Procesando cliente: CLIENTE8
Procesando cliente: CLIENTE9

✅ Archivo 'processingdata_IF.xlsx' generado exitosamente.
